In [ ]:
!pip install autocorrect
# Importējam nepieciešamos moduļus bibliotēkas
# Importējam pandas bibliotēku, lai veiktu manipulācijas ar datiem
import pandas as pd
import json

# Ielādējam datus, kur izmantojam kolonnas RootCauseCategory, FullDescription
data = pd.read_csv('./drive/MyDrive/ColabNotebooks/Data/ModifiedErrors.csv', delimiter="%TAB%")
print(data.head())

# Pārveidojam kategorijas uz skaitļu vērtībām
categories = {
    'Requirements': 0,
    'Architecture': 1,
    'Development': 2,
    'Testing': 3
}
data['Int_RootCauseCategory'] = data['RootCauseCategory'].apply(lambda x: categories[x])

#region Pārveidojam tekstu - transformācija uz mazajiem burtiem
def transformTextToLower():
    def convertToLower(text):
        return text.lower()

    data["Mod_FullDescription"] = data["FullDescription"].apply(lambda x: convertToLower(x))
#endregion

#region Pārveidojam tekstu - izvācām visus HTML tagus
def removeHtmlTags():
    import re
    def removeHtmlTagsFromText(text):
        html_pattern = r'<.*?>'
        without_html = re.sub(pattern=html_pattern, repl=' ', string=text)
        return without_html

    data["Mod_FullDescription"] = data["Mod_FullDescription"].apply(lambda x: removeHtmlTagsFromText(x))
#endregion

#region Pārveidojam tekstu - izdzēšam liekās atstarpes
def removeWhiteSpaces():
    import re
    def removewhiteSpacesFromText(text):
        return re.sub(r'\s+', ' ', text).strip()

    data["Mod_FullDescription"] = data["Mod_FullDescription"].apply(lambda x: removewhiteSpacesFromText(x))
#endregion

#region Pārveidojam tekstu - izdzēšam vietrāžus no teksta
def removeUrlsFromText():
    import re
    def removeUrlsFromText(text):
        url_pattern = r'https?://\S+|www\.\S+'
        without_urls = re.sub(pattern=url_pattern, repl=' ', string=text)
        return without_urls

    data["Mod_FullDescription"] = data["Mod_FullDescription"].apply(lambda x: removeUrlsFromText(x))
#endregion

#region Pārveidojam tekstu - izdzēšam rakstzīmes, kas satur 1 simbolu
def removeSingleChars():
    import re
    def removeSingleCharsFromText(text):
        single_char_pattern = r'\s+[a-zA-Z]\s+'
        without_single_char = re.sub(pattern=single_char_pattern, repl=" ", string=text)
        return without_single_char

    data["Mod_FullDescription"] = data["Mod_FullDescription"].apply(lambda x: removeSingleCharsFromText(x))
#endregion

#region Pārveidojam tekstu - transformētajm no, piemēram, "don't" uz "do not". No īsās uz garo formu
def removeShortWords():
    import contractions
    def removeShortWordsFromText(text):
        expanded_text = []
        for word in text.split():
            expanded_text.append(contractions.fix(word))
        return " ".join(expanded_text)

    data["Mod_FullDescription"] = data["Mod_FullDescription"].apply(lambda x: removeShortWordsFromText(x))
#endregion

#region Pārveidojam tekstu - pāreizrakstības labošana
def autocorrectText():
    from autocorrect import Speller
    import nltk

    nltk.download('punkt_tab')
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('omw-1.4')

    def spellCheckerText(text):
        spellCheckerText.counter += 1
        spellChecker = Speller(lang="en")
        correct_words = []
        for word in nltk.word_tokenize(text):
            correct_word = spellChecker(word)
            correct_words.append(correct_word)
        correct_spell_text = " ".join(correct_words)
        print(spellCheckerText.counter)
        return correct_spell_text
    spellCheckerText.counter = 0
    data["Mod_FullDescription"] = data["Mod_FullDescription"].apply(lambda x: spellCheckerText(x))
#endregion

#region Pārveidojam tekstu - LEMMATIZING - pārveidojam tekstu
def lematizeText():
    import nltk
    from nltk.stem import WordNetLemmatizer
    from nltk import word_tokenize

    nltk.download('punkt_tab')
    nltk.download('punkt')
    nltk.download('wordnet')
    def lemmatizingText(text):
        lemmatizer = WordNetLemmatizer()
        tokens = word_tokenize(text)
        for i in range(len(tokens)):
            lemma_word = lemmatizer.lemmatize(tokens[i])
            tokens[i] = lemma_word
        return " ".join(tokens)

    data["Mod_FullDescription"] = data["Mod_FullDescription"].apply(lambda x: lemmatizingText(x))
#endregion

#region Pārveidojam tekstu - izdzēšam vārdus tādus kā: "the", "and", "is", "in", "for", "it".
def removeStopWord():
    import nltk
    from nltk.corpus import stopwords
    from nltk import word_tokenize

    nltk.download("stopwords")

    def removeStopWordFromText(text):
        removed = []
        stop_words = list(stopwords.words("english"))
        tokens = word_tokenize(text)
        for i in range(len(tokens)):
            if tokens[i] not in stop_words:
                removed.append(tokens[i])
        return " ".join(removed)

    data["Mod_FullDescription"] = data["Mod_FullDescription"].apply(lambda x: removeStopWordFromText(x))
#endregion

#region Pārveidojam tekstu - izdzēšam pieturzīmes
def removePunctuation():
    import string
    def removePunctuationFromText(text):
        return text.translate(str.maketrans('', '', string.punctuation))
    data["Mod_FullDescription"] = data["Mod_FullDescription"].apply(lambda x: removePunctuationFromText(x))
#endregion

transformTextToLower()
removeHtmlTags()
removeUrlsFromText()
removeWhiteSpaces()
removeSingleChars()
removeShortWords()
autocorrectText()
lematizeText()
removeStopWord()
removePunctuation()
data.to_csv('/content/drive/MyDrive/ColabNotebooks/Data/ModifiedTransformed.csv', index=False)
data.to_json('/content/drive/MyDrive/ColabNotebooks/Data/ModifiedTransformed.json', orient="records", indent=4)

<ipython-input-6-7421eae1ecb5>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('./drive/MyDrive/ColabNotebooks/Data/ModifiedErrors.csv', delimiter="%TAB%")
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


    Id    Opening  Reporter VerificationStatus ResolutionStatus  \
0  538  904184243      3791           VERIFIED            FIXED   
1  539  904196016      3794           VERIFIED            FIXED   
2  541  904216125      3794           VERIFIED            FIXED   
3  542  904243888      3794           VERIFIED            FIXED   
4  543  904247176      1069           VERIFIED            FIXED   

  RootCauseCategory                        Components  \
0       Development  HTML Dialogs,UI,Bugzilla-General   
1           Testing       MIMELib,UI,Bugzilla-General   
2       Development         Bugzilla,Bugzilla-General   
3      Requirements         Bugzilla,Bugzilla-General   
4      Requirements         Bugzilla,Bugzilla-General   

                                         Description  \
0   Bugzilla can't search bug lists for a text match   
1  enter_bug.cgi chokes if we have multiple produ...   
2  Sorting the initial query by "assignee" doesn'...   
3  Whine at people when they h

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 